<a href="https://colab.research.google.com/github/GavinHacker/ml_practice/blob/master/recbycontent_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install pymysql

In [0]:
#!pip install git+https://github.com/coreylynch/pyFM

In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import pairwise_distances
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
#df = pd.read_sql_query("select * from movie", get_connection())

In [0]:
#df.shape

In [0]:
#df_exp = df.head(1)

In [0]:
#df_exp

In [0]:
def get_dim_dict(df, dim_name):
  type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
  type_list = [x for l in type_list for x in l]
  def reduce_func(x, y):
    for i in x:
      if i[0] == y[0][0]:
        x.remove(i)
        x.append(((i[0],i[1] + 1)))
        return x
    x.append(y[0])
    return x
  l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_list))
  type_zip = reduce(reduce_func, list(l))
  #type_list = sorted(list(set(type_list)))
  #type_zip = zip(list(range(len(type_list))), type_list)
  #print(len(type_zip))
  #print(type_zip)
  type_dict = {}
  for i in type_zip:
    type_dict[i[0]] = i[1]
  return type_dict

In [0]:
type_dict = get_dim_dict(df, 'type')
actors_dict = get_dim_dict(df, 'actors')
director_dict = get_dim_dict(df, 'director')
trait_dict = get_dim_dict(df, 'trait')

In [0]:
#print(type_dict)
#print(actors_dict)
#print(director_dict)
#print(trait_dict)

In [0]:
#director_dict['吉姆·弗尔']

In [0]:
#actors_dict['马克·莱昂纳蒂']

In [0]:
#df_ = df.drop(['ADD_TIME', 'enable', 'rat', 'id', 'name'], axis=1)

In [0]:
#df_.head()

In [0]:
movie_dict_list = []
for i in df_.index:
  movie_dict = {}
  #type
  for s_type in df_.iloc[i]['type'].split('|'):
    movie_dict[s_type] = 1
  #actors
  for s_actor in df_.iloc[i]['actors'].split('|'):
    if actors_dict[s_actor] < 2:
      movie_dict['other_actor'] = 1
    else:
      movie_dict[s_actor] = 1
  #regios
  movie_dict[df_.iloc[i]['region']] = 1
  #director
  for s_director in df_.iloc[i]['director'].split('|'):
    if director_dict[s_director] < 2:
      movie_dict['other_director'] = 1
    else:
      movie_dict[s_director] = 1
  #trait
  for s_trait in df_.iloc[i]['trait'].split('|'):
    movie_dict[s_trait] = 1
  movie_dict_list.append(movie_dict)

In [0]:
v = DictVectorizer()
X = v.fit_transform(movie_dict_list)

In [0]:
#X.shape

In [0]:
#pd.DataFrame(data=X.toarray(), columns=v.feature_names_).T.head(5)

In [0]:
item_similarity = pairwise_distances(X, metric='cosine')

In [0]:
#item_similarity.shape

In [0]:
compare_index = 3
index = 0
_max_index = 0
_max = 1
for i in item_similarity[compare_index]:
  if i < _max and i != 0:
    _max = i
    _max_index = index
  index = index + 1
print(_max_index, _max)

3634 0.2587506833388987


In [0]:
index_of_sim = _max_index

In [0]:
df.iloc[index_of_sim]
movie_dict_list[index_of_sim]
df_106 = pd.DataFrame(data=X.todense()[index_of_sim], columns=v.feature_names_)
df_0 = pd.DataFrame(data=X.todense()[compare_index], columns=v.feature_names_)
df_diff = pd.concat([df_0, df_106], axis=0, ignore_index=True)
#df_diff.reindex(['0','1'])
df_diff = df_diff.T
print(df_diff[(df_diff[0] != 0) | (df_diff[1] != 0)])

                  0    1
other_actor     1.0  1.0
other_director  1.0  1.0
丹尼·博伊尔          0.0  1.0
佩丽冉卡·曹帕拉        1.0  0.0
剧情              1.0  1.0
励志              1.0  1.0
印度              1.0  1.0
喜剧              1.0  1.0
感人              1.0  1.0
戴夫·帕特尔          0.0  1.0
搞笑              1.0  0.0
文艺              1.0  0.0
沙鲁巴·舒克拉         1.0  1.0
爱情              1.0  1.0
经典              1.0  1.0
青春              0.0  1.0
音乐              1.0  0.0


In [0]:
#df.iloc[index_of_sim]
#movie_dict_list[0]

In [0]:
#df.iloc[compare_index]

In [0]:
#item_similarity.shape

In [0]:
# li_dict = {1:0.1, 2:0.2, 5:0.4, 8:0.3}
# min(li_dict.values())
# dict(filter(lambda x:x[1] != 0.4 ,li_dict.items()))

In [0]:
#df_sim = pd.DataFrame(data=item_similarity)

In [0]:
#df_sim.head(1)

In [0]:
def insert_one_ibmovie(id_,
                      movieid,
                      recmovieid,
                      recmovierat,
                      simrat,
                      time,
                      enable, connection):
  sql = 'insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')' % (id_, movieid, recmovieid, recmovierat, simrat, time, enable)
  print(sql)
  try:
    with connection.cursor() as cursor:
        cout=cursor.execute(sql)
    connection.commit()
  except Exception as e:
    print(e)
    connection.close()

## Test code

In [0]:
# _conn = get_connection()
# insert_one_ibmovie(uuid.uuid4(), '1', '2', '6', '0.1', datetime.datetime.now(), '1', _conn)
# _conn.commit()
# _conn.close()

In [0]:
# df_sim.nsmallest(10, 0)[0]

In [0]:
def process_offline_compute_by_cosdis(rec_per_num):
  rec_per_num = rec_per_num + 1
  connection = get_connection()
  for i in range(0, item_similarity.shape[0]):
    df_sim_p = df_sim.nsmallest(rec_per_num, i)
    df_sim_p = df_sim_p[i]
    movie_id = df.iloc[i]['id']
    time_now = datetime.datetime.now()
    for rec_movie_item in df_sim_p.to_dict().items():
      if rec_movie_item[0] != i:
        rec_movie_index = rec_movie_item[0]
        rec_movie_sim = rec_movie_item[1]
        rec_movie_id = df.iloc[rec_movie_index]['id']
        rec_movie_rat = df.iloc[rec_movie_index]['rat']
        insert_one_ibmovie(id_=uuid.uuid4(), movieid=movie_id, recmovieid=rec_movie_id, recmovierat=rec_movie_rat, simrat=rec_movie_sim, time=time_now, enable='1', connection=connection)
    connection.commit()

In [0]:
process_offline_compute_by_cosdis(rec_per_num=200)

insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('2ea27d62-f4c0-4e50-9de4-bcb0427eeb63','10001432','26375689','6.8','0.2222222222222221','2018-12-20 09:01:36.147226','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('eba4c82f-24d3-4705-aaca-2d2309c83a8b','10001432','2381835','5.7','0.24407105398154572','2018-12-20 09:01:36.147226','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('647c3607-8cb2-4aae-8271-e1de0b5f35ab','10047547','6874456','7.3','0.03175416344814619','2018-12-20 09:01:38.639798','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('7f535096-fe13-4ff6-aaba-a5802d3511c5','10047547','6533054','5.9','0.09630388588493644','2018-12-20 09:01:38.639798','1')
insert into ibmovie (id, movieid, recmovieid, recmovierat, simrat, time, enable) values ('6fa5b62f-dc5d-46cf-8da5-7638355b7489','10342838','1305858','8.0','

KeyboardInterrupt: ignored

In [0]:
# df_sim_3_dict = df_sim_3[4].to_dict()
# for i in df_sim_3_dict.items():
#   print(i)